<a href="https://colab.research.google.com/github/Carlos1729/DGL/blob/main/ClusterGCN_Arxiv_0_47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install watermark

In [2]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu116/repo.html

Looking in links: https://data.dgl.ai/wheels/cu116/repo.html


In [3]:
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html


In [4]:
!pip install torchmetrics==0.11.4

In [5]:
import time

import dgl
import dgl.nn as dglnn

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as MF
from ogb.nodeproppred import DglNodePropPredDataset

In [6]:
class SAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, n_hidden, "mean"))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_hidden, "mean"))
        self.layers.append(dglnn.SAGEConv(n_hidden, n_classes, "mean"))
        self.dropout = nn.Dropout(0.5)

    def forward(self, sg, x):
        h = x
        for l, layer in enumerate(self.layers):
            h = layer(sg, h)
            if l != len(self.layers) - 1:
                h = F.relu(h)
                h = self.dropout(h)
        return h


In [7]:
dataset = dgl.data.AsNodePredDataset(DglNodePropPredDataset("ogbn-arxiv"))
graph = dataset[
    0
]  # already prepares ndata['label'/'train_mask'/'val_mask'/'test_mask']

model = SAGE(graph.ndata["feat"].shape[1], 256, dataset.num_classes).cuda()
opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

num_partitions = 1000
sampler = dgl.dataloading.ClusterGCNSampler(
    graph,
    num_partitions,
    prefetch_ndata=["feat", "label", "train_mask", "val_mask", "test_mask"],
)
# DataLoader for generic dataloading with a graph, a set of indices (any indices, like
# partition IDs here), and a graph sampler.
dataloader = dgl.dataloading.DataLoader(
    graph,
    torch.arange(num_partitions).to("cuda"),
    sampler,
    device="cuda",
    batch_size=100,
    shuffle=True,
    drop_last=False,
    num_workers=0,
    use_uva=True,
)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:10<00:00,  8.06it/s]


Extracting dataset/arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 3342.07it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  7.81it/s]

Saving...


Convert a graph into a bidirected graph: 0.126 seconds, peak memory: 11.549 GB
Construct multi-constraint weights: 0.005 seconds, peak memory: 11.549 GB
Metis partitioning: 1.766 seconds, peak memory: 11.683 GB


In [8]:
durations = []
for epoch in range(10):
    t0 = time.time()
    model.train()
    for it, sg in enumerate(dataloader):
        x = sg.ndata["feat"]
        y = sg.ndata["label"]
        m = sg.ndata["train_mask"].bool()
        y_hat = model(sg, x)
        loss = F.cross_entropy(y_hat[m], y[m])
        opt.zero_grad()
        loss.backward()
        opt.step()
        if it % 20 == 0:
            acc = MF.accuracy(
                y_hat[m],
                y[m],
                task="multiclass",
                num_classes=dataset.num_classes,
            )
            mem = torch.cuda.max_memory_allocated() / 1000000
            print("Loss", loss.item(), "Acc", acc.item(), "GPU Mem", mem, "MB")

    tt = time.time() - t0
    print("Run time for epoch# %d: %.2fs" % (epoch, tt))
    durations.append(tt)

    model.eval()
    with torch.no_grad():
        val_preds, test_preds = [], []
        val_labels, test_labels = [], []
        for it, sg in enumerate(dataloader):
            x = sg.ndata["feat"]
            y = sg.ndata["label"]
            m_val = sg.ndata["val_mask"].bool()
            m_test = sg.ndata["test_mask"].bool()
            y_hat = model(sg, x)
            val_preds.append(y_hat[m_val])
            val_labels.append(y[m_val])
            test_preds.append(y_hat[m_test])
            test_labels.append(y[m_test])
        val_preds = torch.cat(val_preds, 0)
        val_labels = torch.cat(val_labels, 0)
        test_preds = torch.cat(test_preds, 0)
        test_labels = torch.cat(test_labels, 0)
        val_acc = MF.accuracy(
            val_preds,
            val_labels,
            task="multiclass",
            num_classes=dataset.num_classes,
        )
        test_acc = MF.accuracy(
            test_preds,
            test_labels,
            task="multiclass",
            num_classes=dataset.num_classes,
        )
        print("Validation acc:", val_acc.item(), "Test acc:", test_acc.item())

print(
    "Average run time for last %d epochs: %.2fs standard deviation: %.3f"
    % ((epoch - 3), np.mean(durations[4:]), np.std(durations[4:]))
)

Loss 4.094895839691162 Acc 0.04651420935988426 GPU Mem 171.034624 MB
Run time for epoch# 0: 4.88s
Validation acc: 0.2952112555503845 Test acc: 0.250066876411438
Loss 3.042762041091919 Acc 0.2299090027809143 GPU Mem 210.835456 MB
Run time for epoch# 1: 0.29s
Validation acc: 0.2989697754383087 Test acc: 0.24119910597801208
Loss 2.6606216430664062 Acc 0.3371421992778778 GPU Mem 212.059648 MB
Run time for epoch# 2: 0.26s
Validation acc: 0.3717910051345825 Test acc: 0.28345987200737
Loss 2.476545810699463 Acc 0.341734915971756 GPU Mem 212.536832 MB
Run time for epoch# 3: 0.25s
Validation acc: 0.42011475563049316 Test acc: 0.3422216773033142
Loss 2.2883315086364746 Acc 0.40076714754104614 GPU Mem 212.536832 MB
Run time for epoch# 4: 0.25s
Validation acc: 0.44696131348609924 Test acc: 0.37409213185310364
Loss 2.207167148590088 Acc 0.41313469409942627 GPU Mem 212.536832 MB
Run time for epoch# 5: 0.25s
Validation acc: 0.46464645862579346 Test acc: 0.4017241597175598
Loss 2.289783477783203 Acc 0